In [ ]:
!conda create --name geemap_env python=3.8
!conda activate geemap_env

In [ ]:
!conda install -c conda-forge earthengine-api geemap ipywidgets


In [ ]:
!pip install geemap

In [ ]:
!jupyter notebook

In [ ]:
import ee
ee.Initialize()

datasets = ee.data.listAssets({'parent': 'projects/earthengine-public/assets/WorldPop'})
print(datasets)


In [2]:
import ee
ee.Authenticate()
ee.Initialize()


Enter verification code:  4/1AUJR-x7QpmCQrHLcBGKDk-io2KgiRmuyDCLQlZK0NAfqOpTzZJnwmJsaq_A



Successfully saved authorization token.


In [ ]:
# Import required libraries
import ee
import geemap
import ipywidgets as widgets
import matplotlib.pyplot as plt

# Initialize Earth Engine
ee.Initialize()

# Define population datasets
datasets = {
    "GPW v4.11 Population Density": {
        "id": "CIESIN/GPWv411/GPW_Population_Density",
        "resolution": "30 arc-seconds (~1 km)",
        "type": "Unconstrained"
    },
    "LandScan Global": {
        "id": "projects/sat-io/open-datasets/ORNL/LANDSCAN_GLOBAL",
        "resolution": "1 km",
        "type": "Unconstrained"
    },
    "WorldPop 100m": {
        "id": "WorldPop/GP/100m/pop",
        "resolution": "100m",
        "type": "Unconstrained"
    },
    "GHS-POP": {
        "id": "JRC/GHSL/P2016/POP_GPW_GLOBE_V1",
        "resolution": "250m",
        "type": "Constrained"
    },
    "WorldPop 1km": {
        "id": "WorldPop/GP/1km/pop",
        "resolution": "1km",
        "type": "Unconstrained"
    },
    "GPW v4.11 Urban Population": {
        "id": "CIESIN/GPWv411/GPW_Population_Urban",
        "resolution": "30 arc-seconds (~1 km)",
        "type": "Unconstrained"
    }
}

# Class bins matching legend
bin_thresholds = [1, 6, 26, 51, 101, 501, 2501, 5001, 185000]
legend_labels = ['1-5', '6-25', '26-50', '51-100', '101-500', '501-2500', '2501-5000', '5001-185000']
legend_colors = ['#ffffcc', '#fff775', '#fed976', '#feb24c', '#fd8d3c', '#f03b20', '#bd0026', '#800026']

# Function to load population dataset
def load_population_dataset(dataset_id):
    dataset = ee.ImageCollection(dataset_id)
    try:
        dataset = dataset.filter(ee.Filter.date('2000-01-01', '2024-12-31')).median()
    except:
        dataset = dataset.median()
    return dataset

# Load all datasets
population_layers = {name: load_population_dataset(info["id"]) for name, info in datasets.items()}

# Create the interactive map
Map = geemap.Map()
Map.add_basemap("OpenStreetMap")

# Function to classify image into bins
def classify_image(image):
    bins = bin_thresholds
    classified = image.gt(bins[0]).And(image.lte(bins[1])).multiply(1)
    for i in range(1, len(bins)-1):
        classified = classified.where(image.gt(bins[i]).And(image.lte(bins[i+1])), i+1)
    return classified

# Visualization parameters
vis_params = {
    'min': 1,
    'max': len(legend_colors),
    'palette': legend_colors
}

# Function to get and display population data
def get_population_data(b):
    print("Processing the selected area...")

    drawn_features = Map.draw_features
    if not drawn_features:
        print("No region selected. Please draw a polygon on the map first.")
        return

    feature = ee.Geometry(drawn_features[-1].geometry())
    Map.centerObject(feature, zoom=10)
    
    # Remove all non-base layers (population layers), keep only draw layer
    Map.layers = Map.layers[1:]
    Map.add_basemap("OpenStreetMap")  # Re-add OSM so it's placed at the bottom

    population_values = {}

    for name, dataset in population_layers.items():
        clipped = dataset.clip(feature)

        count = clipped.reduceRegion(
            reducer=ee.Reducer.count(),
            geometry=feature,
            scale=1000,
            maxPixels=1e12
        ).getInfo()

        if not count:
            print(f"No data for {name} in AOI.")
            continue

        # ✅ Mask zero and nodata pixels
        masked = clipped.updateMask(clipped.gt(0))
        classified = classify_image(masked)
        Map.addLayer(classified, vis_params, f"{name} (classified)")

        mean_val = clipped.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=feature,
            scale=1000,
            maxPixels=1e12
        ).getInfo()

        population_values[name] = mean_val.get('population', None)

    if all(v is not None for v in population_values.values()):
        plot_population_distribution(population_values)
    else:
        print("Some datasets had missing values.")


# Plot population values
def plot_population_distribution(population_values):
    labels = list(population_values.keys())
    values = [population_values[name] for name in labels]

    plt.figure(figsize=(10, 6))
    plt.bar(labels, values, color='skyblue')
    plt.xlabel('Population Datasets')
    plt.ylabel('Population per Pixel (Mean)')
    plt.title('Population Density per Pixel for Each Dataset')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

# Add legend to the map
def add_legend():
    Map.add_legend(
        title="Population Count (estimate)",
        labels=legend_labels,
        colors=legend_colors,
        position='bottomright'
    )

# Add draw control and legend
Map.add_draw_control()
add_legend()

# Add the Get Data button
button = widgets.Button(description="Get Population Data")
button.on_click(get_population_data)

# Display map and button
display(Map)
display(button)


In [ ]:
! GPW allocates population uniformly within administrative units, so when you zoom in, it looks like blocky shapes — not actual population clusters.

In [ ]:
import ee
ee.Authenticate()
ee.Initialize()


In [ ]:
Dataset	Why it differs
GPW	Uniformly spreads census population — no smart modeling.(Lightly Modeled)
LandScan	Models likely day/night distribution using multiple datasets.(heavily modeled) 
WorldPop	High-res smart modeling based on land cover and socioeconomic data. (heavily modeled) 
GHS-POP	Only places people where buildings exist constrained model. (Lightly Modeled)
Graph of the sum (each dataset)

In [ ]:
# Import required libraries
import ee
import geemap
import ipywidgets as widgets
import matplotlib.pyplot as plt

# Initialize Earth Engine
ee.Initialize()

# Define datasets with their band names
datasets = {
    "GPW v4.11 Population Density": {
        "id": "CIESIN/GPWv411/GPW_Population_Density",
        "band": "population_density",
        "years": [2015, 2020]
    },
    "LandScan Global": {
        "id": "projects/sat-io/open-datasets/ORNL/LANDSCAN_GLOBAL",
        "band": "population",
        "years": [2014, 2015, 2018, 2019, 2020]
    },
    "WorldPop 100m": {
        "id": "WorldPop/GP/100m/pop",
        "band": "population",
        "years": [2015, 2020]
    },
    "GHS-POP": {
        "id": "JRC/GHSL/P2016/POP_GPW_GLOBE_V1",
        "band": "population",
        "years": [2015]
    },
    "WorldPop 1km": {
        "id": "WorldPop/GP/1km/pop",
        "band": "population",
        "years": [2015, 2020]
    }
}

# Years to allow user to select from
years = [2014, 2015, 2018, 2019, 2020]

# Visualization bins
bin_thresholds = [1, 6, 26, 51, 101, 501, 2501, 5001, 185000]
legend_labels = ['1-5', '6-25', '26-50', '51-100', '101-500', '501-2500', '2501-5000', '5001-185000']
legend_colors = ['#ffffcc', '#fff775', '#fed976', '#feb24c', '#fd8d3c', '#f03b20', '#bd0026', '#800026']

vis_params = {
    'min': 1,
    'max': len(legend_colors),
    'palette': legend_colors
}

# Function to classify image into bins
def classify_image(image):
    bins = bin_thresholds
    classified = image.gt(bins[0]).And(image.lte(bins[1])).multiply(1)
    for i in range(1, len(bins) - 1):
        classified = classified.where(image.gt(bins[i]).And(image.lte(bins[i + 1])), i + 1)
    return classified

# Load dataset image by year
def load_dataset_by_year(dataset_id, band, year):
    try:
        collection = ee.ImageCollection(dataset_id).filterDate(f'{year}-01-01', f'{year}-12-31')
        image = collection.select(band).sort('system:time_start', False).first()
        return ee.Image(image)
    except Exception as e:
        print(f"Error loading {dataset_id} for {year}: {e}")
        return None

# Initialize map
Map = geemap.Map()
Map.add_basemap("OpenStreetMap")
Map.add_draw_control()

# Dataset dropdown
dataset_dropdown = widgets.Dropdown(
    options=list(datasets.keys()),
    value="GPW v4.11 Population Density",
    description="Dataset:"
)

# Year dropdown
year_dropdown = widgets.Dropdown(
    options=years,
    value=2020,
    description="Year:"
)

# Add legend
def add_legend():
    Map.add_legend(
        title="Population Density",
        labels=legend_labels,
        colors=legend_colors,
        position='bottomright'
    )

add_legend()

# Button function
def get_population_data(b):
    print("Processing the selected area...")

    drawn_features = Map.draw_features
    if not drawn_features:
        print("⚠️ Please draw a polygon on the map.")
        return

    geometry = ee.Geometry(drawn_features[-1].geometry())
    Map.centerObject(geometry, zoom=9)

    # Clear existing layers except base
    Map.layers = Map.layers[:1]

    selected_dataset = dataset_dropdown.value
    selected_year = year_dropdown.value

    dataset_info = datasets[selected_dataset]
    
    if selected_year not in dataset_info['years']:
        print(f"⚠️ No data available for {selected_dataset} in {selected_year}. Try a different year.")
        return

    image = load_dataset_by_year(dataset_info["id"], dataset_info["band"], selected_year)
    if image is None:
        print("⚠️ Image not found.")
        return

    clipped = image.clip(geometry)
    masked = clipped.updateMask(clipped.gt(0))
    classified = classify_image(masked)

    Map.addLayer(classified, vis_params, f"{selected_dataset} ({selected_year})")

    stats = clipped.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=geometry,
        scale=1000,
        maxPixels=1e12
    ).getInfo()

    pop_mean = stats.get(dataset_info["band"], None)
    if pop_mean is not None:
        print(f"✅ {selected_dataset} ({selected_year}) - Mean population per pixel: {pop_mean:.2f}")
    else:
        print("⚠️ No population data in the selected region.")

# Button widget
button = widgets.Button(description="Get Population Data")
button.on_click(get_population_data)

# UI
ui = widgets.VBox([dataset_dropdown, year_dropdown, button])
display(ui)
display(Map)


In [ ]:
# Import required libraries
import ee
import geemap
import ipywidgets as widgets
import matplotlib.pyplot as plt

# Initialize Earth Engine
ee.Initialize()

# Define datasets with their band names
datasets = {
    "GPW v4.11 Population Density": {
        "id": "CIESIN/GPWv411/GPW_Population_Density",
        "band": "population_density",
        "years": [2015, 2020]
    },
    "LandScan Global": {
        "id": "projects/sat-io/open-datasets/ORNL/LANDSCAN_GLOBAL",
        "band": "population",
        "years": [2014, 2015,2016, 2017, 2018, 2019, 2020]
    },
    "WorldPop 100m": {
        "id": "WorldPop/GP/100m/pop",
        "band": "population",
        "years": [2014,2015,2016,2017,2018,2019,2020,2021]
    },
    "GHS-POP": {
        "id": "JRC/GHSL/P2016/POP_GPW_GLOBE_V1",
        "band": "population",
        "years": [2015]
    },
    "WorldPop 1km": {
        "id": "WorldPop/GP/1km/pop",
        "band": "population",
        "years": [2015, 2020]
    }
}

# Years to allow user to select from
years = [2014, 2015, 2018, 2019, 2020]

# Visualization bins
bin_thresholds = [1, 6, 26, 51, 101, 501, 2501, 5001, 185000]
legend_labels = ['1-5', '6-25', '26-50', '51-100', '101-500', '501-2500', '2501-5000', '5001-185000']
legend_colors = ['#ffffcc', '#fff775', '#fed976', '#feb24c', '#fd8d3c', '#f03b20', '#bd0026', '#800026']

vis_params = {
    'min': 1,
    'max': len(legend_colors),
    'palette': legend_colors
}

# Function to classify image into bins
def classify_image(image):
    bins = bin_thresholds
    classified = image.gt(bins[0]).And(image.lte(bins[1])).multiply(1)
    for i in range(1, len(bins) - 1):
        classified = classified.where(image.gt(bins[i]).And(image.lte(bins[i + 1])), i + 1)
    return classified

# Load dataset image by year
def load_dataset_by_year(dataset_id, band, year):
    try:
        if "LANDSCAN_GLOBAL" in dataset_id:
            # Get image by year string in ID
            collection = ee.ImageCollection(dataset_id)
            image = collection.filter(ee.Filter.stringContains("system:index", str(year))).first()
        else:
            collection = ee.ImageCollection(dataset_id).filterDate(f'{year}-01-01', f'{year}-12-31')
            image = collection.select(band).sort('system:time_start', False).first()

        return ee.Image(image)
    except Exception as e:
        print(f"Error loading {dataset_id} for {year}: {e}")
        return None

# Initialize map
Map = geemap.Map()
Map.add_basemap("OpenStreetMap")
Map.add_draw_control()

# Dataset dropdown
dataset_dropdown = widgets.Dropdown(
    options=list(datasets.keys()),
    value="GPW v4.11 Population Density",
    description="Dataset:"
)

# Year dropdown
year_dropdown = widgets.Dropdown(
    options=years,
    value=2020,
    description="Year:"
)

# Add legend
def add_legend():
    Map.add_legend(
        title="Population Density",
        labels=legend_labels,
        colors=legend_colors,
        position='bottomright'
    )

add_legend()

# Button function
def get_population_data(b):
    print("Processing the selected area...")

    drawn_features = Map.draw_features
    if not drawn_features:
        print("⚠️ Please draw a polygon on the map.")
        return

    geometry = ee.Geometry(drawn_features[-1].geometry())
    Map.centerObject(geometry, zoom=9)

    # Clear existing layers except base
    Map.layers = Map.layers[:1]

    selected_dataset = dataset_dropdown.value
    selected_year = year_dropdown.value

    dataset_info = datasets[selected_dataset]
    
    if selected_year not in dataset_info['years']:
        print(f"⚠️ No data available for {selected_dataset} in {selected_year}. Try a different year.")
        return

    image = load_dataset_by_year(dataset_info["id"], dataset_info["band"], selected_year)
    if image is None:
        print("⚠️ Image not found.")
        return

    clipped = image.clip(geometry)
    masked = clipped.updateMask(clipped.gt(0))
    classified = classify_image(masked)

    Map.addLayer(classified, vis_params, f"{selected_dataset} ({selected_year})")

    # Instead of reducing to mean, we just calculate the total population
    stats = clipped.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=geometry,
        scale=1000,
        maxPixels=1e12
    ).getInfo()

    pop_total = stats.get(dataset_info["band"], None)
    if pop_total is not None:
        print(f"✅ {selected_dataset} ({selected_year}) - Total population: {pop_total:.2f}")
    else:
        print("⚠️ No population data in the selected region.")

# Button widget
button = widgets.Button(description="Get Population Data")
button.on_click(get_population_data)

# UI
ui = widgets.VBox([dataset_dropdown, year_dropdown, button])
display(ui)
display(Map)


In [ ]:
# Import required libraries
import ee
import geemap
import ipywidgets as widgets
import matplotlib.pyplot as plt

# Initialize Earth Engine
ee.Initialize()

# Define datasets with their band names
datasets = {
    "GPW v4.11 Population Density": {
        "id": "CIESIN/GPWv411/GPW_Population_Density",
        "band": "population_density",
        "years": [2015, 2020]
    },
    "LandScan Global": {
        "id": "projects/sat-io/open-datasets/ORNL/LANDSCAN_GLOBAL",
        "band": "population",
        "years": [2014, 2015, 2016, 2017, 2018, 2019, 2020]
    },
    "WorldPop 100m": {
        "id": "WorldPop/GP/100m/pop",
        "band": "population",
        "years": [2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021]
    },
    "GHS-POP": {
        "id": "JRC/GHSL/P2016/POP_GPW_GLOBE_V1",
        "band": "population",
        "years": [2015]
    },
    "WorldPop 1km": {
        "id": "WorldPop/GP/1km/pop",
        "band": "population",
        "years": [2015, 2020]
    }
}

# Years to allow user to select from
years = [2014, 2015, 2018, 2019, 2020]

# Visualization bins
bin_thresholds = [1, 6, 26, 51, 101, 501, 2501, 5001, 185000]
legend_labels = ['1-5', '6-25', '26-50', '51-100', '101-500', '501-2500', '2501-5000', '5001-185000']
legend_colors = ['#ffffcc', '#fff775', '#fed976', '#feb24c', '#fd8d3c', '#f03b20', '#bd0026', '#800026']

vis_params = {
    'min': 1,
    'max': len(legend_colors),
    'palette': legend_colors
}

# Function to classify image into bins
def classify_image(image):
    bins = bin_thresholds
    classified = image.gt(bins[0]).And(image.lte(bins[1])).multiply(1)
    for i in range(1, len(bins) - 1):
        classified = classified.where(image.gt(bins[i]).And(image.lte(bins[i + 1])), i + 1)
    return classified

# Load dataset image by year
def load_dataset_by_year(dataset_id, band, year):
    try:
        if "LANDSCAN_GLOBAL" in dataset_id:
            # Get image by year string in ID
            collection = ee.ImageCollection(dataset_id)
            image = collection.filter(ee.Filter.stringContains("system:index", str(year))).first()
        elif "WorldPop" in dataset_id:
            # Get image by year for WorldPop dataset
            collection = ee.ImageCollection(dataset_id).filter(ee.Filter.stringContains("system:index", str(year)))
            image = collection.first()  # Get the first image that matches the year
        else:
            # For other datasets (like GPW and GHS-POP)
            collection = ee.ImageCollection(dataset_id).filterDate(f'{year}-01-01', f'{year}-12-31')
            image = collection.select(band).sort('system:time_start', False).first()

        return ee.Image(image)
    except Exception as e:
        print(f"Error loading {dataset_id} for {year}: {e}")
        return None

# Initialize map
Map = geemap.Map()
Map.add_basemap("OpenStreetMap")
Map.add_draw_control()

# Dataset dropdown
dataset_dropdown = widgets.Dropdown(
    options=list(datasets.keys()),
    value="GPW v4.11 Population Density",
    description="Dataset:"
)

# Year dropdown
year_dropdown = widgets.Dropdown(
    options=years,
    value=2020,
    description="Year:"
)

# Add legend
def add_legend():
    Map.add_legend(
        title="Population Density",
        labels=legend_labels,
        colors=legend_colors,
        position='bottomright'
    )

add_legend()

# Button function
def get_population_data(b):
    print("Processing the selected area...")

    drawn_features = Map.draw_features
    if not drawn_features:
        print("⚠️ Please draw a polygon on the map.")
        return

    geometry = ee.Geometry(drawn_features[-1].geometry())
    Map.centerObject(geometry, zoom=9)

    # Clear existing layers except base
    Map.layers = Map.layers[:1]

    selected_dataset = dataset_dropdown.value
    selected_year = year_dropdown.value

    dataset_info = datasets[selected_dataset]
    
    if selected_year not in dataset_info['years']:
        print(f"⚠️ No data available for {selected_dataset} in {selected_year}. Try a different year.")
        return

    image = load_dataset_by_year(dataset_info["id"], dataset_info["band"], selected_year)
    if image is None:
        print("⚠️ Image not found.")
        return

    clipped = image.clip(geometry)
    masked = clipped.updateMask(clipped.gt(0))
    classified = classify_image(masked)

    Map.addLayer(classified, vis_params, f"{selected_dataset} ({selected_year})")

    # Instead of reducing to mean, we just calculate the total population
    stats = clipped.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=geometry,
        scale=1000,
        maxPixels=1e12
    ).getInfo()

    pop_total = stats.get(dataset_info["band"], None)
    if pop_total is not None:
        print(f"✅ {selected_dataset} ({selected_year}) - Total population: {pop_total:.2f}")
    else:
        print("⚠️ No population data in the selected region.")

# Button widget
button = widgets.Button(description="Get Population Data")
button.on_click(get_population_data)

# UI
ui = widgets.VBox([dataset_dropdown, year_dropdown, button])
display(ui)
display(Map)


In [ ]:
# Import required libraries
import ee
import geemap
import ipywidgets as widgets
import matplotlib.pyplot as plt

# Initialize Earth Engine
ee.Initialize()

# Define datasets with their band names
datasets = {
    "GPW v4.11 Population Density": {
        "id": "CIESIN/GPWv411/GPW_UNWPP-Adjusted_Population_Density",
        "band": "unwpp-adjusted_population_density",
        "years": [2000, 2005, 2010, 2015, 2020]
    },
    "LandScan Global": {
        "id": "projects/sat-io/open-datasets/ORNL/LANDSCAN_GLOBAL",
        "band": "population",
        "years": [2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021]
    },
    "WorldPop 100m": {
        "id": "WorldPop/GP/100m/pop",
        "band": "population",
        "years": list(range(2000, 2021))  # Updated year range
    },
    "GHS-POP": {
        "id": "JRC/GHSL/P2016/POP_GPW_GLOBE_V1",
        "band": "population",
        "years": [2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021]
    },
    "WorldPop 100m Unconstrained 2020": {
        "id": "WorldPop/UNadj/100m/pop",
        "band": "unconstrained",
        "years": [2020]
    },
    "GHSL Global Population Surfaces (P2023A)": {
        "id": "JRC/GHSL/P2023A/GHS_POP",
        "band": "population_count",  # Band name in GHSL dataset
        "years": list(range(1975, 2031))  # Dataset includes years from 1975 to 2030
    }
}

# Years to allow user to select from
years = [2000, 2005, 2010, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021]

# Visualization bins
bin_thresholds = [1, 6, 26, 51, 101, 501, 2501, 5001, 185000]
legend_labels = ['1-5', '6-25', '26-50', '51-100', '101-500', '501-2500', '2501-5000', '5001-185000']
legend_colors = ['#ffffcc', '#fff775', '#fed976', '#feb24c', '#fd8d3c', '#f03b20', '#bd0026', '#800026']

vis_params = {
    'min': 1,
    'max': len(legend_colors),
    'palette': legend_colors
}

# Function to classify image into bins
def classify_image(image):
    bins = bin_thresholds
    classified = image.gt(bins[0]).And(image.lte(bins[1])).multiply(1)
    for i in range(1, len(bins) - 1):
        classified = classified.where(image.gt(bins[i]).And(image.lte(bins[i + 1])), i + 1)
    return classified


def load_dataset_by_year(dataset_id, band, year):
    try:
        if "LANDSCAN_GLOBAL" in dataset_id:
            collection = ee.ImageCollection(dataset_id)
            image = collection.filter(ee.Filter.stringContains("system:index", str(year))).first()

        elif dataset_id == "WorldPop/GP/100m/pop":
            collection = ee.ImageCollection(dataset_id).filterDate(f'{year}-01-01', f'{year+1}-01-01')
            image = collection.first()
            # Auto-detect available band
            band_names = image.bandNames().getInfo()
            if 'population' in band_names:
                image = image.select('population')
            else:
                print(f"⚠️ No 'population' band found for WorldPop {year}.")
                return None

        elif dataset_id == "WorldPop/UNadj/100m/pop":
            collection = ee.ImageCollection(dataset_id).filterDate(f'{year}-01-01', f'{year+1}-01-01')
            image = collection.select('unconstrained').first()

        elif dataset_id == "JRC/GHSL/P2023A/GHS_POP":
            collection = ee.ImageCollection(dataset_id).filterDate(f'{year}-01-01', f'{year+1}-01-01')
            image = collection.select('population_count').first()

        else:
            collection = ee.ImageCollection(dataset_id).filterDate(f'{year}-01-01', f'{year+1}-01-01')
            image = collection.select(band).sort('system:time_start', False).first()

        return ee.Image(image)
    except Exception as e:
        print(f"Error loading {dataset_id} for {year}: {e}")
        return None


# Initialize map
Map = geemap.Map()
Map.add_basemap("OpenStreetMap")
Map.add_draw_control()

# Dataset dropdown
dataset_dropdown = widgets.Dropdown(
    options=list(datasets.keys()),
    value="GPW v4.11 Population Density",
    description="Dataset:"
)

# Year dropdown
year_dropdown = widgets.Dropdown(
    options=years,
    value=2020,
    description="Year:"
)

# Add legend
def add_legend():
    Map.add_legend(
        title="Population Density",
        labels=legend_labels,
        colors=legend_colors,
        position='bottomright'
    )

add_legend()

# Button function
def get_population_data(b):
    print("Processing the selected area...")

    drawn_features = Map.draw_features
    if not drawn_features:
        print("⚠️ Please draw a polygon on the map.")
        return

    geometry = ee.Geometry(drawn_features[-1].geometry())
    Map.centerObject(geometry, zoom=9)

    # Clear existing layers except base
    Map.layers = Map.layers[:1]

    selected_dataset = dataset_dropdown.value
    selected_year = year_dropdown.value

    dataset_info = datasets[selected_dataset]

    if selected_year not in dataset_info['years']:
        print(f"⚠️ No data available for {selected_dataset} in {selected_year}. Try a different year.")
        return

    image = load_dataset_by_year(dataset_info["id"], dataset_info["band"], selected_year)
    if image is None:
        print("⚠️ Image not found.")
        return

    clipped = image.clip(geometry)
    masked = clipped.updateMask(clipped.gt(0))
    classified = classify_image(masked)

    Map.addLayer(classified, vis_params, f"{selected_dataset} ({selected_year})")

    # Instead of reducing to mean, we just calculate the total population
    stats = clipped.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=geometry,
        scale=1000,
        maxPixels=1e12
    ).getInfo()

    pop_total = stats.get(dataset_info["band"], None)
    if pop_total is not None:
        print(f"✅ {selected_dataset} ({selected_year}) - Total population: {pop_total:.2f}")
    else:
        print("⚠️ No population data in the selected region.")

# Button widget
button = widgets.Button(description="Get Population Data")
button.on_click(get_population_data)

# UI
ui = widgets.VBox([dataset_dropdown, year_dropdown, button])
display(ui)
display(Map)


In [ ]:
import ee
import geemap
import ipywidgets as widgets

# Initialize Earth Engine
ee.Initialize()

# Create output widget
output = widgets.Output()

# Dataset dictionary with GRUMP and other population datasets
datasets = {
    "CIESIN/GPWv411/GPW_UNWPP-Adjusted_Population_Density": {
        "label": "GPW v4.11 Population Density",
        "band": "unwpp-adjusted_population_density",
        "years": [2000, 2005, 2010, 2015, 2020]
    },
    "LandScan_Global": {
        "label": "LandScan Global",
        "band": "b1",
        "years": [2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021],
        "dynamic_id": "projects/sat-io/open-datasets/ORNL/LANDSCAN_GLOBAL/landscan-global-{year}"
    },
    "WorldPop/GP/100m/pop": {
        "label": "WorldPop 100m",
        "band": "population",
        "years": [2010, 2015, 2020]
    },
    "JRC/GHSL/P2016/POP_GPW_GLOBE_V1": {
        "label": "GHS-POP",
        "band": "population",
        "years": [2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021]

    },
    "JRC/GHSL/P2023A/GHS_POP": {
        "label": "GHSL Global Population Surfaces (P2023A)",
        "band": "population_count",
        "years": list(range(1975, 2031))
    },
}

# Visualization parameters
bin_thresholds = [1, 6, 26, 51, 101, 501, 2501, 5001, 185000]
legend_labels = ['1-5', '6-25', '26-50', '51-100', '101-500', '501-2500', '2501-5000', '5001-185000']
legend_colors = ['#ffffcc', '#fff775', '#fed976', '#feb24c', '#fd8d3c', '#f03b20', '#bd0026', '#800026']

vis_params = {
    'min': 1,
    'max': len(legend_colors),
    'palette': legend_colors
}

def classify_image(image):
    bins = bin_thresholds
    classified = image.gt(bins[0]).And(image.lte(bins[1])).multiply(1)
    for i in range(1, len(bins) - 1):
        classified = classified.where(image.gt(bins[i]).And(image.lte(bins[i + 1])), i + 1)
    return classified

def load_dataset_by_year(dataset_id, band, year, country=None):
    try:
        if dataset_id == "LandScan_Global":
            image_id = datasets[dataset_id]["dynamic_id"].format(year=year)
            image = ee.Image(image_id).select(band)
        else:
            collection = ee.ImageCollection(dataset_id).filterDate(f'{year}-01-01', f'{year+1}-01-01')
            if country:
                collection = collection.filterBounds(country)
            image = collection.select(band).first()
        return ee.Image(image)
    except Exception as e:
        print(f"Error loading {dataset_id} for {year}: {e}")
        return None

# Initialize map
Map = geemap.Map()
Map.add_basemap("OpenStreetMap")
Map.add_draw_control()

# Dropdown widgets
dataset_dropdown = widgets.Dropdown(
    options=[(v["label"], k) for k, v in datasets.items()],
    value="WorldPop/GP/100m/pop",
    description="Dataset:"
)

year_dropdown = widgets.Dropdown(
    options=datasets["WorldPop/GP/100m/pop"]["years"],
    value=2020,
    description="Year:"
)

# Update year options when dataset changes
def update_year_dropdown(change):
    dataset_id = change["new"]
    year_dropdown.options = datasets[dataset_id]["years"]
    year_dropdown.value = datasets[dataset_id]["years"][-1]

dataset_dropdown.observe(update_year_dropdown, names="value")

# Add legend
def add_legend():
    Map.add_legend(
        title="Population Count",
        labels=legend_labels,
        colors=legend_colors,
        position='bottomright'
    )

add_legend()

# Countries collection
countries = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017")

# Button handler
def get_population_data(b):
    with output:
        output.clear_output()
        print("🔍 Processing the selected area...")

        drawn_features = Map.draw_features
        if not drawn_features:
            print("⚠️ Please draw a polygon on the map.")
            return

        geometry = ee.Geometry(drawn_features[-1].geometry())
        Map.centerObject(geometry, zoom=9)
        Map.layers = Map.layers[:1]

        dataset_id = dataset_dropdown.value
        dataset_info = datasets[dataset_id]
        selected_year = year_dropdown.value

        if selected_year not in dataset_info["years"]:
            print(f"⚠️ {dataset_info['label']} does not support year {selected_year}.")
            return

        if dataset_id == "WorldPop/GP/100m/pop":
            matching_countries = countries.filterBounds(geometry)
            num_countries = len(matching_countries.getInfo()['features'])
            print(f"✅ Number of countries found: {num_countries}")

            country_images = []

            for country in matching_countries.getInfo()['features']:
                country_geometry = ee.Geometry(country['geometry'])
                image = load_dataset_by_year(dataset_id, dataset_info["band"], selected_year, country=country_geometry)
                if image:
                    country_images.append(image)

            if country_images:
                combined_image = ee.ImageCollection(country_images).sum()
                clipped = combined_image.clip(geometry)
                masked = clipped.updateMask(clipped.gt(0))
                classified = classify_image(masked)

                Map.addLayer(classified, vis_params, f"{dataset_info['label']} ({selected_year})")

                stats = combined_image.reduceRegion(
                    reducer=ee.Reducer.sum(),
                    geometry=geometry,
                    scale=1000,
                    maxPixels=1e12
                ).getInfo()

                pop_total = stats.get(dataset_info["band"], None)
                if pop_total is not None:
                    print(f"✅ {dataset_info['label']} ({selected_year}) - Total population: {pop_total:.2f}")
                else:
                    print("⚠️ No population data in the selected region.")
            else:
                print("⚠️ No valid population data for the selected countries.")

        else:
            image = load_dataset_by_year(dataset_id, dataset_info["band"], selected_year)
            if image is None:
                print("⚠️ Failed to load image.")
                return

            clipped = image.clip(geometry)
            masked = clipped.updateMask(clipped.gt(0))
            classified = classify_image(masked)

            Map.addLayer(classified, vis_params, f"{dataset_info['label']} ({selected_year})")

            stats = image.reduceRegion(
                reducer=ee.Reducer.sum(),
                geometry=geometry,
                scale=1000,
                maxPixels=1e12
            ).getInfo()

            pop_total = stats.get(dataset_info["band"], None)
            if pop_total is not None:
                print(f"✅ {dataset_info['label']} ({selected_year}) - Total population: {pop_total:.2f}")
            else:
                print("⚠️ No population data in the selected region.")

# Button widget
button = widgets.Button(description="Get Population Data")
button.on_click(get_population_data)

# Display UI
ui = widgets.VBox([dataset_dropdown, year_dropdown, button, output])
display(ui)
display(Map)


In [ ]:
import geemap
print(geemap.__version__)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.dates import DateFormatter

def prepare_data(df):
    df = df.copy()
    df['Date'] = pd.to_datetime(df['Date'])
    df['Close'] = df['Close/Last'].str.replace('$', '').astype(float)
    df['High'] = df['High'].str.replace('$', '').astype(float)
    df['Low'] = df['Low'].str.replace('$', '').astype(float)
    return df.sort_values('Date')

def create_multi_stock_animation(amd_df, nvidia_df, intel_df):
    # Prepare data
    amd_data = prepare_data(amd_df)
    nvidia_data = prepare_data(nvidia_df)
    intel_data = prepare_data(intel_df)
    
    fig, ax = plt.subplots(figsize=(12, 6))

    # Overall min/max for axes limits
    min_date = min(amd_data['Date'].min(), nvidia_data['Date'].min(), intel_data['Date'].min())
    max_date = max(amd_data['Date'].max(), nvidia_data['Date'].max(), intel_data['Date'].max())
    
    min_price = min(amd_data['Low'].min(), nvidia_data['Low'].min(), intel_data['Low'].min()) * 0.95
    max_price = max(amd_data['High'].max(), nvidia_data['High'].max(), intel_data['High'].max()) * 1.05
    
    ax.set_xlim(min_date, max_date)
    ax.set_ylim(min_price, max_price)
    
    # Lines
    amd_line, = ax.plot([], [], lw=2, color='#ED1C24', label='AMD')
    nvidia_line, = ax.plot([], [], lw=2, color='#76B900', label='NVIDIA')
    intel_line, = ax.plot([], [], lw=2, color='#0071C5', label='Intel')
    
    # Text labels
    amd_text = ax.text(0, 0, '', color='white', fontweight='bold',
                      bbox=dict(facecolor='#ED1C24', alpha=0.7, edgecolor='none', pad=3))
    nvidia_text = ax.text(0, 0, '', color='white', fontweight='bold',
                         bbox=dict(facecolor='#76B900', alpha=0.7, edgecolor='none', pad=3))
    intel_text = ax.text(0, 0, '', color='white', fontweight='bold',
                        bbox=dict(facecolor='#0071C5', alpha=0.7, edgecolor='none', pad=3))
    
    ax.legend(loc='upper left')
    ax.set_title('Semiconductor Stock Prices Comparison', pad=15)
    ax.set_xlabel('Date')
    ax.set_ylabel('Price ($)')
    
    # Date formatting
    ax.xaxis.set_major_formatter(DateFormatter('%Y-%m-%d'))
    plt.xticks(rotation=45)
    ax.grid(True, linestyle='--', alpha=0.7)
    plt.subplots_adjust(bottom=0.2)
    
    def animate(frame):
        # Update lines
        amd_x = amd_data['Date'].iloc[:frame]
        amd_y = amd_data['Close'].iloc[:frame]
        amd_line.set_data(amd_x, amd_y)
        
        nvidia_x = nvidia_data['Date'].iloc[:frame]
        nvidia_y = nvidia_data['Close'].iloc[:frame]
        nvidia_line.set_data(nvidia_x, nvidia_y)
        
        intel_x = intel_data['Date'].iloc[:frame]
        intel_y = intel_data['Close'].iloc[:frame]
        intel_line.set_data(intel_x, intel_y)
        
        # Update labels with slight vertical offset
        if len(amd_y) > 0:
            amd_text.set_position((amd_x.iloc[-1], amd_y.iloc[-1] * 1.01))
            amd_text.set_text(f'AMD ${amd_y.iloc[-1]:.2f}')
        if len(nvidia_y) > 0:
            nvidia_text.set_position((nvidia_x.iloc[-1], nvidia_y.iloc[-1] * 1.01))
            nvidia_text.set_text(f'NVIDIA ${nvidia_y.iloc[-1]:.2f}')
        if len(intel_y) > 0:
            intel_text.set_position((intel_x.iloc[-1], intel_y.iloc[-1] * 1.01))
            intel_text.set_text(f'Intel ${intel_y.iloc[-1]:.2f}')
        
        # Return updated artists for blitting
        return amd_line, nvidia_line, intel_line, amd_text, nvidia_text, intel_text
    
    frames = max(len(amd_data), len(nvidia_data), len(intel_data))
    anim = animation.FuncAnimation(fig, animate, frames=frames, interval=50, blit=True, repeat=False)
    
    return anim, fig

if __name__ == "__main__":
    # Load your CSV data here:
    amd_df = pd.read_csv('amd.csv')
    nvidia_df = pd.read_csv('nvidia.csv')
    intel_df = pd.read_csv('intel.csv')
    
    anim, fig = create_multi_stock_animation(amd_df, nvidia_df, intel_df)
    
    # Save MP4 (needs ffmpeg)
    anim.save('stock_animation.mp4', writer='ffmpeg', fps=30)
    
    # Save GIF (needs pillow)
    anim.save('stock_animation.gif', writer='pillow', fps=30)
    
    plt.show()


In [ ]:
pip install pandas matplotlib pillow


In [ ]:
pip install rasterio matplotlib numpy


In [ ]:
import os

folder = r'C:\Users\JM505 Computers\Downloads\Landscan\landscan-global-2020-assets'
files = [f for f in os.listdir(folder) if f.lower().endswith('.tif')]

print("TIFF files found in folder:")
for f in files:
    print(f)


In [ ]:
import rasterio
import matplotlib.pyplot as plt
import numpy as np

for filename in files:
    filepath = os.path.join(folder, filename)
    year = ''.join(filter(str.isdigit, filename))  # Extract digits as year
    
    with rasterio.open(filepath) as src:
        data = src.read(1)
        data = np.where(data == 0, np.nan, data)  # Mask zero population
        
        plt.figure(figsize=(12, 8))
        plt.title(f'LandScan Population - {year}')
        img = plt.imshow(data, cmap='viridis')
        plt.colorbar(img, label='Population Count')
        plt.axis('off')
        plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.plot([1, 2, 3], [4, 5, 6])
plt.title("Test Plot")
plt.show()


In [ ]:
%matplotlib inline


In [11]:
import rasterio
from rasterio.enums import Resampling
import matplotlib.pyplot as plt
import numpy as np
import os
import matplotlib.animation as animation
from IPython.display import HTML

base_parent_folder = r'C:\Users\JM505 Computers\Downloads\Landscan'

# Only years 2016 to 2020 for animation
year_folders = [
    'landscan-global-2016-assets',
    'landscan-global-2017-assets',
    'landscan-global-2018-assets',
    'landscan-global-2019-assets',
    'landscan-global-2020-assets',
]

scale_factor = 10  # downsample factor, adjust as needed

# List to hold downsampled data arrays and years
frames = []
years = []

for folder_name in year_folders:
    folder_path = os.path.join(base_parent_folder, folder_name)
    year = ''.join(filter(str.isdigit, folder_name))
    
    # Find TIFF files in folder (assuming one per folder)
    tiff_files = [f for f in os.listdir(folder_path) if f.lower().endswith('.tif')]
    if not tiff_files:
        print(f"No TIFF found in {folder_path}")
        continue
    
    filepath = os.path.join(folder_path, tiff_files[0])  # Take first TIFF
    
    with rasterio.open(filepath) as src:
        new_width = src.width // scale_factor
        new_height = src.height // scale_factor
        
        data = src.read(
            1,
            out_shape=(1, new_height, new_width),
            resampling=Resampling.average
        )
        data = np.where(data == 0, np.nan, data)
        
    frames.append(data)
    years.append(year)

print(f"Loaded frames for years: {years}")

# Prepare animation plot
fig, ax = plt.subplots(figsize=(12, 8))
img = ax.imshow(frames[0], cmap='viridis')
cbar = fig.colorbar(img, ax=ax, label='Population Count')
ax.axis('off')
title = ax.set_title(f'LandScan Population - {years[0]}')

def update(frame_idx):
    img.set_data(frames[frame_idx])
    title.set_text(f'LandScan Population - {years[frame_idx]}')
    return img, title

anim = animation.FuncAnimation(
    fig, update,
    frames=len(frames),
    interval=1000,  # 1 second per frame
    repeat=True
)

# Save animation as GIF
output_gif = os.path.join(base_parent_folder, 'LandScan_2016_2020_population.gif')
anim.save(output_gif, writer='pillow', fps=1)
print(f"Saved animation to {output_gif}")

plt.close(fig)  # close plot to avoid duplicate display

# Display inline in notebook
HTML(anim.to_jshtml())


ModuleNotFoundError: No module named 'rasterio'

In [ ]:
pip install geopandas rasterio matplotlib numpy imageio


In [ ]:
C:\Users\JM505 Computers\Downloads\Landscan\Pakistan_LandScan_Animated_Cleaned.gif


In [ ]:
import geopandas as gpd
import rasterio
from rasterio.mask import mask
from skimage.transform import resize
import matplotlib.pyplot as plt
import numpy as np
import os
import imageio
from IPython.display import Image, display

# Step 1: Load Pakistan shapefile
shapefile_path = r'C:\Users\JM505 Computers\Downloads\PAK_adm\PAK_adm.shp'
pakistan = gpd.read_file(shapefile_path).to_crs(epsg=4326)

# Step 2: Define LandScan folders by year
base_folder = r'C:\Users\JM505 Computers\Downloads\Landscan'
year_map = {
    '2000': 'landscan-global-2000-assets',
    '2005': 'landscan-global-2005-assets',
    '2010': 'landscan-global-2010-assets',
    '2015': 'landscan-global-2015-assets',
    '2020': 'landscan-global-2020-assets',
    '2023': 'landscan-global-2023-assets',
}

scale_factor = 10
frames = []
years = []

# Step 3: Clip each raster to Pakistan and downsample
for year, folder in year_map.items():
    folder_path = os.path.join(base_folder, folder)
    tiff_files = [f for f in os.listdir(folder_path) if f.endswith('.tif') and 'colorized' not in f]
    if not tiff_files:
        print(f"⚠️ No valid TIFF found for {year}")
        continue

    filepath = os.path.join(folder_path, tiff_files[0])
    with rasterio.open(filepath) as src:
        # Clip to Pakistan boundary
        pakistan_proj = pakistan.to_crs(src.crs)
        geojson = [f["geometry"] for f in pakistan_proj.__geo_interface__["features"]]
        clipped, _ = mask(src, geojson, crop=True)
        clipped = clipped[0]

        # Downsample
        new_height = clipped.shape[0] // scale_factor
        new_width = clipped.shape[1] // scale_factor
        resized = resize(clipped, (new_height, new_width), anti_aliasing=True, preserve_range=True)
        resized = np.where(resized == 0, np.nan, resized)

        frames.append(resized)
        years.append(year)

print(f"✅ Loaded valid population data for years: {years}")

# Step 4: Determine percentile stretch for contrast
all_data = np.concatenate([f.flatten() for f in frames])
vmin = np.nanpercentile(all_data, 2)
vmax = np.nanpercentile(all_data, 98)

# Step 5: Create animated GIF
gif_path = os.path.join(base_folder, 'Pakistan_LandScan_REAL.gif')
fig, ax = plt.subplots(figsize=(8, 6), facecolor='white')
images = []

for i in range(len(frames)):
    ax.clear()
    fig.patch.set_facecolor('white')
    ax.set_facecolor('white')

    img = ax.imshow(frames[i], cmap='inferno', vmin=vmin, vmax=vmax)
    ax.set_title(f"Pakistan Population Count - {years[i]}", fontsize=12, pad=10, color='black')
    ax.axis('off')

    cbar = fig.colorbar(img, ax=ax, shrink=0.6, orientation='vertical', pad=0.01)
    cbar.set_label('Population Count per Pixel', fontsize=9, color='black', labelpad=8)
    cbar.ax.tick_params(labelsize=8, colors='black')
    cbar.outline.set_edgecolor('black')

    temp_file = f'temp_{years[i]}.png'
    plt.savefig(temp_file, dpi=150, bbox_inches='tight', facecolor='white')
    images.append(imageio.imread(temp_file))
    os.remove(temp_file)

imageio.mimsave(gif_path, images, duration=1.5)
print(f"✅ Animation saved to: {gif_path}")

plt.close(fig)
display(Image(filename=gif_path))


In [ ]:
pip install scikit-image


In [ ]:
pip install geopandas rasterio scikit-image imageio matplotlib


In [ ]:
import matplotlib.pyplot as plt
import rasterio

filepath = r'C:\Users\JM505 Computers\Downloads\Landscan\landscan-global-2020-assets\landscan-global-2020.tif'

with rasterio.open(filepath) as src:
    data = src.read(1)
    print(f"Min: {np.min(data)}, Max: {np.max(data)}, Mean: {np.mean(data)}")

    plt.figure(figsize=(10, 6))
    plt.imshow(data, cmap='inferno', vmin=1, vmax=np.percentile(data, 99))
    plt.colorbar(label="Population Count")
    plt.title("Raw LandScan 2020")
    plt.axis('off')
    plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np

# Path to your CSV file (update as needed)
data_path = r'C:\Users\JM505 Computers\Downloads\API_SP.POP.TOTL_DS2_en_csv_v2_85220\API_SP.POP.TOTL_DS2_en_csv_v2_85220.csv'

# Load data skipping metadata rows
df = pd.read_csv(data_path, skiprows=4)

# Top 5 populous Asian countries
top5_countries = ['India', 'China', 'Indonesia', 'Pakistan', 'Bangladesh']

# Filter these countries
df_top5 = df[df['Country Name'].isin(top5_countries)]

# Select year columns 2010 to 2023
years = [str(y) for y in range(2010, 2024)]
df_top5 = df_top5[['Country Name'] + years].reset_index(drop=True)

# Sort by 2023 population descending to rank colors
df_top5 = df_top5.sort_values(by='2023', ascending=False).reset_index(drop=True)

# Define discrete color palette for 5 countries (from highest to lowest population)
color_palette = ['darkred', 'lightcoral', 'orange', 'green', 'yellow']

# Plot setup
bar_width = 0.15
x = np.arange(len(years))

plt.figure(figsize=(14, 8))

# Plot bars with assigned colors based on rank
for i, country in enumerate(df_top5['Country Name']):
    plt.bar(x + i * bar_width, df_top5.loc[i, years], width=bar_width, label=country, color=color_palette[i])

# Labels and title
plt.xlabel('Year')
plt.ylabel('Population')
plt.title('Population Trends (2010-2023) for Top 5 Most Populous Asian Countries')

# X-axis ticks centered under groups
plt.xticks(x + bar_width*2, years, rotation=45)

# Format y-axis with SI suffixes
def si_formatter(x, pos):
    if x >= 1_000_000_000:
        return f'{x/1_000_000_000:.1f}B'
    elif x >= 1_000_000:
        return f'{x/1_000_000:.0f}M'
    elif x >= 1_000:
        return f'{x/1_000:.0f}K'
    else:
        return f'{int(x)}'

plt.gca().yaxis.set_major_formatter(ticker.FuncFormatter(si_formatter))

plt.legend(loc='upper left')
plt.grid(axis='y')
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.animation as animation
from IPython.display import HTML

# Load the data
data_path = r'C:\Users\JM505 Computers\Downloads\API_SP.POP.TOTL_DS2_en_csv_v2_85220\API_SP.POP.TOTL_DS2_en_csv_v2_85220.csv'
df = pd.read_csv(data_path, skiprows=4)

# Select top 5 countries and years
top5_countries = ['India', 'China', 'Indonesia', 'Pakistan', 'Bangladesh']
years = [str(y) for y in range(2010, 2024)]

df_top5 = df[df['Country Name'].isin(top5_countries)]
df_top5 = df_top5[['Country Name'] + years].set_index('Country Name')
df_top5 = df_top5.astype(float).T  # Transpose for animation (years as rows)
df_top5.index.name = 'Year'

# Color map
colors = {
    'India': 'darkred',
    'China': 'lightcoral',
    'Indonesia': 'orange',
    'Pakistan': 'green',
    'Bangladesh': 'yellow'
}

# Set up the plot
fig, ax = plt.subplots(figsize=(10, 6))

def si_formatter(x, pos):
    if x >= 1e9:
        return f'{x/1e9:.1f}B'
    elif x >= 1e6:
        return f'{x/1e6:.0f}M'
    elif x >= 1e3:
        return f'{x/1e3:.0f}K'
    return str(int(x))

def update(frame):
    ax.clear()
    year = df_top5.index[frame]
    values = df_top5.loc[year].sort_values(ascending=True)
    ax.barh(values.index, values.values, color=[colors[c] for c in values.index])
    ax.set_xlim(0, df_top5.max().max() * 1.05)
    ax.set_title(f'Population of Top 5 Asian Countries in {year}', fontsize=16)
    ax.set_xlabel('Population')
    ax.xaxis.set_major_formatter(ticker.FuncFormatter(si_formatter))
    for i, (country, value) in enumerate(values.items()):
        ax.text(value, i, f'{value:,.0f}', va='center', ha='left', fontsize=9)

# Animate and display
anim = animation.FuncAnimation(fig, update, frames=len(df_top5.index), interval=1000, repeat=False)

HTML(anim.to_jshtml())


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.animation as animation
from IPython.display import HTML
import numpy as np

# Load the data
data_path = r'C:\Users\JM505 Computers\Downloads\API_SP.POP.TOTL_DS2_en_csv_v2_85220\API_SP.POP.TOTL_DS2_en_csv_v2_85220.csv'
df = pd.read_csv(data_path, skiprows=4)

# Select year columns 2010 to 2023
years = [str(y) for y in range(2010, 2024)]

# Prepare dataframe: Countries as columns, years as rows
df_all = df[['Country Name'] + years].set_index('Country Name').astype(float).T
df_all.index.name = 'Year'

# To keep the plot readable, show only top N countries by population each year
top_n = 10  # change this number to show more or fewer countries

# Set up the plot
fig, ax = plt.subplots(figsize=(12, 8))

def si_formatter(x, pos):
    if x >= 1e9:
        return f'{x/1e9:.1f}B'
    elif x >= 1e6:
        return f'{x/1e6:.0f}M'
    elif x >= 1e3:
        return f'{x/1e3:.0f}K'
    return str(int(x))

def update(frame):
    ax.clear()
    year = df_all.index[frame]
    data_year = df_all.loc[year].dropna()
    top_countries = data_year.sort_values(ascending=False).head(top_n)
    ax.barh(top_countries.index[::-1], top_countries.values[::-1], color=plt.cm.tab20.colors)
    ax.set_xlim(0, df_all.max().max() * 1.05)
    ax.set_title(f'Top {top_n} Most Populous Countries in {year}', fontsize=18)
    ax.set_xlabel('Population')
    ax.xaxis.set_major_formatter(ticker.FuncFormatter(si_formatter))
    ax.grid(axis='x', linestyle='--', alpha=0.7)
    # Annotate population values
    for i, (country, value) in enumerate(zip(top_countries.index[::-1], top_countries.values[::-1])):
        ax.text(value * 1.01, i, f'{int(value):,}', va='center', ha='left', fontsize=9)

# Create animation object
anim = animation.FuncAnimation(fig, update, frames=len(df_all.index), interval=1000, repeat=False)

HTML(anim.to_jshtml())


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.animation as animation
from IPython.display import HTML
import numpy as np

# Load data
data_path = r'C:\Users\JM505 Computers\Downloads\API_SP.POP.TOTL_DS2_en_csv_v2_85220\API_SP.POP.TOTL_DS2_en_csv_v2_85220.csv'
df = pd.read_csv(data_path, skiprows=4)

years = [str(y) for y in range(2010, 2024)]
df = df[['Country Name'] + years].set_index('Country Name').astype(float).T
df.index.name = 'Year'

# Calculate total world population for each year
df['World Total'] = df.sum(axis=1)

top_n = 10  # top countries shown per year

# Assign fixed colors for consistency (using tab20 colormap)
color_map = plt.cm.get_cmap('tab20', top_n)
colors = [color_map(i) for i in range(top_n)]
others_color = 'grey'

fig, ax = plt.subplots(figsize=(14, 9))

def si_formatter(x, pos):
    if x >= 1e9:
        return f'{x/1e9:.1f}B'
    elif x >= 1e6:
        return f'{x/1e6:.0f}M'
    elif x >= 1e3:
        return f'{x/1e3:.0f}K'
    return str(int(x))

def update(frame):
    ax.clear()
    year = df.index[frame]
    data_year = df.loc[year].drop('World Total').dropna()
    
    top_countries = data_year.sort_values(ascending=False).head(top_n)
    others_pop = data_year.sum() - top_countries.sum()
    
    # Add "Others" to data
    top_countries['Others'] = others_pop
    
    # Bars sorted descending
    top_countries = top_countries.sort_values()
    
    bars = ax.barh(top_countries.index, top_countries.values, color=colors + [others_color])
    
    ax.set_xlim(0, df['World Total'].max()*1.05)
    ax.set_title(f'Top {top_n} Countries by Population + Others in {year}', fontsize=20)
    ax.set_xlabel('Population')
    ax.xaxis.set_major_formatter(ticker.FuncFormatter(si_formatter))
    ax.grid(axis='x', linestyle='--', alpha=0.6)
    
    # Annotate bar values
    for bar in bars:
        width = bar.get_width()
        ax.text(width * 1.01, bar.get_y() + bar.get_height()/2,
                f'{int(width):,}', va='center', ha='left', fontsize=10)
    
    # Show total world population as annotation
    ax.text(0.99, 0.15, f'World Total: {int(df.loc[year,"World Total"]):,}',
            transform=ax.transAxes, ha='right', fontsize=14, fontweight='bold',
            bbox=dict(facecolor='white', alpha=0.7, edgecolor='gray'))

anim = animation.FuncAnimation(fig, update, frames=len(df.index), interval=1000, repeat=False)

HTML(anim.to_jshtml())


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.animation as animation
from IPython.display import HTML

# Load World Bank data
data_path = r'C:\Users\JM505 Computers\Downloads\API_SP.POP.TOTL_DS2_en_csv_v2_85220\API_SP.POP.TOTL_DS2_en_csv_v2_85220.csv'
df = pd.read_csv(data_path, skiprows=4)

years = [str(y) for y in range(2010, 2024)]
df = df[['Country Name'] + years]

# Simple country to continent mapping (example, add all needed countries)
country_to_continent = {
    'India': 'Asia',
    'China': 'Asia',
    'Indonesia': 'Asia',
    'Pakistan': 'Asia',
    'Bangladesh': 'Asia',
    'United States': 'North America',
    'Brazil': 'South America',
    'Nigeria': 'Africa',
    'Germany': 'Europe',
    'Australia': 'Oceania',
    # Add more countries here or load from a file
}

# Map countries to continents, remove countries without continent info
df['Continent'] = df['Country Name'].map(country_to_continent)
df = df.dropna(subset=['Continent'])

# Convert years columns to numeric for aggregation
df[years] = df[years].apply(pd.to_numeric, errors='coerce')

# Aggregate population per continent per year
continent_pop = df.groupby('Continent')[years].sum().T
continent_pop.index.name = 'Year'

# Calculate world total for annotation
continent_pop['World Total'] = continent_pop.sum(axis=1)

# Colors for continents
continent_colors = {
    'Asia': 'orange',
    'Africa': 'green',
    'Europe': 'blue',
    'North America': 'red',
    'South America': 'purple',
    'Oceania': 'cyan',
}

fig, ax = plt.subplots(figsize=(12, 8))

def si_formatter(x, pos):
    if x >= 1e9:
        return f'{x/1e9:.1f}B'
    elif x >= 1e6:
        return f'{x/1e6:.0f}M'
    elif x >= 1e3:
        return f'{x/1e3:.0f}K'
    return str(int(x))

def update(frame):
    ax.clear()
    year = continent_pop.index[frame]
    data = continent_pop.loc[year].drop('World Total').sort_values()
    
    bars = ax.barh(data.index, data.values, color=[continent_colors[c] for c in data.index])
    
    ax.set_xlim(0, continent_pop['World Total'].max() * 1.05)
    ax.set_title(f'Population by Continent in {year}', fontsize=18)
    ax.set_xlabel('Population')
    ax.xaxis.set_major_formatter(ticker.FuncFormatter(si_formatter))
    ax.grid(axis='x', linestyle='--', alpha=0.7)
    
    # Annotate bars
    for bar in bars:
        width = bar.get_width()
        ax.text(width * 1.01, bar.get_y() + bar.get_height()/2,
                f'{int(width):,}', va='center', ha='left', fontsize=12)
    
    # World total annotation
    ax.text(0.99, 0.15, f'World Total: {int(continent_pop.loc[year,"World Total"]):,}',
            transform=ax.transAxes, ha='right', fontsize=14, fontweight='bold',
            bbox=dict(facecolor='white', alpha=0.7, edgecolor='gray'))

anim = animation.FuncAnimation(fig, update, frames=len(continent_pop.index), interval=1000, repeat=False)

HTML(anim.to_jshtml())


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.animation as animation
from IPython.display import HTML
import numpy as np

# Load the data
data_path = r'C:\Users\JM505 Computers\Downloads\API_SP.POP.TOTL_DS2_en_csv_v2_85220\API_SP.POP.TOTL_DS2_en_csv_v2_85220.csv'
df = pd.read_csv(data_path, skiprows=4)

# Select year columns 2010 to 2023
years = [str(y) for y in range(2010, 2024)]

# Filter for the top 10 populous countries you provided
top10_countries = [
    'India', 'China', 'United States', 'Indonesia', 'Pakistan',
    'Nigeria', 'Brazil', 'Bangladesh', 'Russian Federation', 'Mexico'
]

# Filter dataframe for those countries + year columns, set index by country
df_top10 = df[df['Country Name'].isin(top10_countries)][['Country Name'] + years]

# Reformat: countries as columns, years as rows
df_all = df_top10.set_index('Country Name').T.astype(float)
df_all.index.name = 'Year'

# Optional: reorder columns by estimated 2023 population descending
df_all = df_all[top10_countries]

# Set up the plot
fig, ax = plt.subplots(figsize=(12, 8))

def si_formatter(x, pos):
    if x >= 1e9:
        return f'{x/1e9:.1f}B'
    elif x >= 1e6:
        return f'{x/1e6:.0f}M'
    elif x >= 1e3:
        return f'{x/1e3:.0f}K'
    return str(int(x))

def update(frame):
    ax.clear()
    year = df_all.index[frame]
    data_year = df_all.loc[year].dropna()
    # Sort descending to show largest populations at the top
    top_countries = data_year.sort_values(ascending=True)
    
    ax.barh(top_countries.index, top_countries.values, color=plt.cm.tab10.colors)
    ax.set_xlim(0, df_all.max().max() * 1.05)
    ax.set_title(f'Top 10 Most Populous Countries in {year}', fontsize=18)
    ax.set_xlabel('Population')
    ax.xaxis.set_major_formatter(ticker.FuncFormatter(si_formatter))
    ax.grid(axis='x', linestyle='--', alpha=0.7)
    ax.set_ylabel('Country')

    # Annotate population values on bars
    for i, (country, value) in enumerate(zip(top_countries.index, top_countries.values)):
        ax.text(value * 1.01, i, f'{int(value):,}', va='center', ha='left', fontsize=10)

# Create animation object, keep anim assigned to avoid deletion
anim = animation.FuncAnimation(fig, update, frames=len(df_all.index), interval=1000, repeat=False)

# Display animation inline in Jupyter
HTML(anim.to_jshtml())


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.animation as animation
from IPython.display import HTML
import numpy as np

# Load the data
data_path = r'C:\Users\JM505 Computers\Downloads\API_SP.POP.TOTL_DS2_en_csv_v2_85220\API_SP.POP.TOTL_DS2_en_csv_v2_85220.csv'
df = pd.read_csv(data_path, skiprows=4)

# Select year columns 2010 to 2023
years = [str(y) for y in range(2010, 2024)]

# Filter for the top 10 populous countries you provided
top10_countries = [
    'India', 'China', 'United States', 'Indonesia', 'Pakistan',
    'Nigeria', 'Brazil', 'Bangladesh', 'Russian Federation', 'Mexico'
]

# Filter dataframe for those countries + year columns, set index by country
df_top10 = df[df['Country Name'].isin(top10_countries)][['Country Name'] + years]

# Reformat: countries as columns, years as rows
df_all = df_top10.set_index('Country Name').T.astype(float)
df_all.index.name = 'Year'

# Optional: reorder columns by estimated 2023 population descending
df_all = df_all[top10_countries]

# Custom color ramp (ordered from highest to lowest)
custom_colors = [
    '#ff5733',  # red-orange
    '#ffbd33',  # orange-yellow
    '#dbff33',  # lime
    '#75ff33',  # light green
    '#33ff57',  # green
    '#33ffbd',  # mint
    '#89752a',  # khaki brown
    '#6d892a',  # olive green
    '#3e892a',  # dark green
    '#2a8945',  # forest green
    
]

# Set up the plot
fig, ax = plt.subplots(figsize=(12, 8))

def si_formatter(x, pos):
    if x >= 1e9:
        return f'{x/1e9:.1f}B'
    elif x >= 1e6:
        return f'{x/1e6:.0f}M'
    elif x >= 1e3:
        return f'{x/1e3:.0f}K'
    return str(int(x))

def update(frame):
    ax.clear()
    year = df_all.index[frame]
    data_year = df_all.loc[year].dropna()
    # Sort descending to show largest populations at the top
    top_countries = data_year.sort_values(ascending=True)
    
    # Assign colors based on rank (highest gets first color)
    sorted_vals = data_year.sort_values(ascending=False)
    color_map = {country: custom_colors[i] for i, country in enumerate(sorted_vals.index)}
    bar_colors = [color_map[country] for country in top_countries.index]

    ax.barh(top_countries.index, top_countries.values, color=bar_colors)
    ax.set_xlim(0, df_all.max().max() * 1.05)
    ax.set_title(f'Top 10 Most Populous Countries in {year}', fontsize=18)
    ax.set_xlabel('Population')
    ax.xaxis.set_major_formatter(ticker.FuncFormatter(si_formatter))
    ax.grid(axis='x', linestyle='--', alpha=0.7)
    ax.set_ylabel('Country')

    # Annotate population values on bars
    for i, (country, value) in enumerate(zip(top_countries.index, top_countries.values)):
        ax.text(value * 1.01, i, f'{int(value):,}', va='center', ha='left', fontsize=10)

# Create animation object, keep anim assigned to avoid deletion
anim = animation.FuncAnimation(fig, update, frames=len(df_all.index), interval=1000, repeat=False)

# Display animation inline in Jupyter
HTML(anim.to_jshtml())


In [ ]:
pip install earthengine-api geemap ipywidgets


In [ ]:
import ipywidgets as widgets
widgets.IntSlider()


In [ ]:
import ipywidgets as widgets
widgets.IntSlider()


In [5]:
import ee
import geemap
import ipywidgets as widgets

# Initialize Earth Engine
ee.Initialize()

# Create output widget
output = widgets.Output()

# Dataset dictionary
datasets = {
    "CIESIN/GPWv411/GPW_UNWPP-Adjusted_Population_Density": {
        "label": "GPW v4.11 Population Density",
        "band": "unwpp-adjusted_population_density",
        "years": [2000, 2005, 2010, 2015, 2020]
    },
    "LandScan_Global": {
        "label": "LandScan Global",
        "band": "b1",
        "years": [2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021],
        "dynamic_id": "projects/sat-io/open-datasets/ORNL/LANDSCAN_GLOBAL/landscan-global-{year}"
    },
    "WorldPop/GP/100m/pop": {
        "label": "WorldPop 100m",
        "band": "population",
        "years": [2010, 2015, 2020]
    },
    "JRC/GHSL/P2016/POP_GPW_GLOBE_V1": {
        "label": "GHS-POP",
        "band": "population",  # Note: we handle band selection specially
        "years": [2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021]
    },
    "JRC/GHSL/P2023A/GHS_POP": {
        "label": "GHSL Global Population Surfaces (P2023A)",
        "band": "population_count",
        "years": list(range(1975, 2031))
    },
}

# Visualization parameters
bin_thresholds = [1, 6, 26, 51, 101, 501, 2501, 5001, 185000]
legend_labels = ['1-5', '6-25', '26-50', '51-100', '101-500', '501-2500', '2501-5000', '5001-185000']
legend_colors = ['#ffffcc', '#fff775', '#fed976', '#feb24c', '#fd8d3c', '#f03b20', '#bd0026', '#800026']

vis_params = {
    'min': 1,
    'max': len(legend_colors),
    'palette': legend_colors
}

def classify_image(image):
    bins = bin_thresholds
    classified = image.gt(bins[0]).And(image.lte(bins[1])).multiply(1)
    for i in range(1, len(bins) - 1):
        classified = classified.where(image.gt(bins[i]).And(image.lte(bins[i + 1])), i + 1)
    return classified

def load_dataset_by_year(dataset_id, band, year, country=None):
    try:
        if dataset_id == "LandScan_Global":
            image_id = datasets[dataset_id]["dynamic_id"].format(year=year)
            image = ee.Image(image_id).select(band)

        elif dataset_id == "JRC/GHSL/P2016/POP_GPW_GLOBE_V1":
            # This dataset has bands named like population_2015, population_2016 etc.
            band_name = f'population_{year}'
            image = ee.Image(dataset_id).select(band_name)
            if image.bandNames().size().getInfo() == 0:
                print(f"⚠️ Band {band_name} not found in {dataset_id}")
                return None

        elif dataset_id == "JRC/GHSL/P2023A/GHS_POP":
            # This is an ImageCollection with images per year filtered by 'year' property
            collection = ee.ImageCollection(dataset_id).filter(ee.Filter.eq('year', year))
            image = collection.first()
            if image is None:
                print(f"⚠️ No image found for {dataset_id} year {year}")
                return None

        else:
            # For other datasets (collections), filter by date and select first image
            collection = ee.ImageCollection(dataset_id).filterDate(f'{year}-01-01', f'{year+1}-01-01')
            if country:
                collection = collection.filterBounds(country)
            image = collection.select(band).first()

        if image is None:
            print(f"⚠️ No image found for {dataset_id} year {year}")
            return None

        return ee.Image(image)

    except Exception as e:
        print(f"Error loading {dataset_id} for {year}: {e}")
        return None

# Initialize map
Map = geemap.Map()
Map.add_basemap("OpenStreetMap")
Map.add_draw_control()

# Dropdown for dataset
dataset_dropdown = widgets.Dropdown(
    options=[(v["label"], k) for k, v in datasets.items()],
    value="WorldPop/GP/100m/pop",
    description="Dataset:"
)

# SelectionSlider for year
year_slider = widgets.SelectionSlider(
    options=datasets["WorldPop/GP/100m/pop"]["years"],
    value=2020,
    description="Year:",
    layout=widgets.Layout(width="95%"),
    continuous_update=False
)

# Add legend
def add_legend():
    Map.add_legend(
        title="Population Count",
        labels=legend_labels,
        colors=legend_colors,
        position='bottomright'
    )

add_legend()

# Countries collection
countries = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017")

# Button handler
def get_population_data(b):
    with output:
        output.clear_output()
        print("🔍 Processing the selected area...")

        drawn_features = Map.draw_features
        if not drawn_features:
            print("⚠️ Please draw a polygon on the map.")
            return

        geometry = ee.Geometry(drawn_features[-1].geometry())
        Map.centerObject(geometry, zoom=9)
        Map.layers = Map.layers[:1]

        dataset_id = dataset_dropdown.value
        dataset_info = datasets[dataset_id]
        selected_year = year_slider.value

        if selected_year not in dataset_info["years"]:
            print(f"⚠️ {dataset_info['label']} does not support year {selected_year}.")
            return

        if dataset_id == "WorldPop/GP/100m/pop":
            matching_countries = countries.filterBounds(geometry)
            num_countries = len(matching_countries.getInfo()['features'])
            print(f"✅ Number of countries found: {num_countries}")

            country_images = []

            for country in matching_countries.getInfo()['features']:
                country_geometry = ee.Geometry(country['geometry'])
                image = load_dataset_by_year(dataset_id, dataset_info["band"], selected_year, country=country_geometry)
                if image:
                    country_images.append(image)

            if not country_images:
                print("⚠️ No valid population data for the selected countries.")
                return

            combined_image = ee.ImageCollection(country_images).sum()
            clipped = combined_image.clip(geometry)
            masked = clipped.updateMask(clipped.gt(0))
            classified = classify_image(masked)

            Map.addLayer(classified, vis_params, f"{dataset_info['label']} ({selected_year})")

            stats = combined_image.reduceRegion(
                reducer=ee.Reducer.sum(),
                geometry=geometry,
                scale=1000,
                maxPixels=1e12
            ).getInfo()

            pop_total = stats.get(dataset_info["band"], None)
            if pop_total is not None:
                print(f"✅ {dataset_info['label']} ({selected_year}) - Total population: {pop_total:.2f}")
            else:
                print("⚠️ No population data in the selected region.")

        else:
            image = load_dataset_by_year(dataset_id, dataset_info["band"], selected_year)
            if image is None:
                print("⚠️ Failed to load image.")
                return

            clipped = image.clip(geometry)
            masked = clipped.updateMask(clipped.gt(0))
            classified = classify_image(masked)

            Map.addLayer(classified, vis_params, f"{dataset_info['label']} ({selected_year})")

            stats = image.reduceRegion(
                reducer=ee.Reducer.sum(),
                geometry=geometry,
                scale=1000,
                maxPixels=1e12
            ).getInfo()

            pop_total = stats.get(dataset_info["band"], None)
            if pop_total is not None:
                print(f"✅ {dataset_info['label']} ({selected_year}) - Total population: {pop_total:.2f}")
            else:
                print("⚠️ No population data in the selected region.")

# Button widget
button = widgets.Button(description="Get Population Data")
button.on_click(get_population_data)

# Slider event connection
def on_slider_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        get_population_data(None)

year_slider.observe(on_slider_change)

# Dataset dropdown handler to update year slider
def update_slider(change):
    dataset_id = change["new"]
    years = datasets[dataset_id]["years"]
    year_slider.options = years
    year_slider.value = years[-1]

dataset_dropdown.observe(update_slider, names="value")

# Initial year slider setup
initial_years = datasets[dataset_dropdown.value]["years"]
year_slider.options = initial_years
year_slider.value = initial_years[-1]

# Display UI and Map
ui = widgets.VBox([dataset_dropdown, year_slider, button, output])
display(ui)
display(Map)


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [4]:
import ee
import geemap
import ipywidgets as widgets

# Initialize Earth Engine
ee.Initialize()

# Create output widget
output = widgets.Output()

# Dataset dictionary with GHSL datasets removed
datasets = {
    "CIESIN/GPWv411/GPW_UNWPP-Adjusted_Population_Density": {
        "label": "GPW v4.11 Population Density",
        "band": "unwpp-adjusted_population_density",
        "years": [2000, 2005, 2010, 2015, 2020]
    },
    "LandScan_Global": {
        "label": "LandScan Global",
        "band": "b1",
        "years": [2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021],
        "dynamic_id": "projects/sat-io/open-datasets/ORNL/LANDSCAN_GLOBAL/landscan-global-{year}"
    },
    "WorldPop/GP/100m/pop": {
        "label": "WorldPop 100m",
        "band": "population",
        "years": [2010, 2015, 2020]
    }
}

# Visualization parameters
bin_thresholds = [1, 6, 26, 51, 101, 501, 2501, 5001, 185000]
legend_labels = ['1-5', '6-25', '26-50', '51-100', '101-500', '501-2500', '2501-5000', '5001-185000']
legend_colors = ['#ffffcc', '#fff775', '#fed976', '#feb24c', '#fd8d3c', '#f03b20', '#bd0026', '#800026']

vis_params = {
    'min': 1,
    'max': len(legend_colors),
    'palette': legend_colors
}

def classify_image(image):
    bins = bin_thresholds
    classified = image.gt(bins[0]).And(image.lte(bins[1])).multiply(1)
    for i in range(1, len(bins) - 1):
        classified = classified.where(image.gt(bins[i]).And(image.lte(bins[i + 1])), i + 1)
    return classified

def load_dataset_by_year(dataset_id, band, year, country=None):
    try:
        if dataset_id == "LandScan_Global":
            image_id = datasets[dataset_id]["dynamic_id"].format(year=year)
            image = ee.Image(image_id).select(band)

        else:
            collection = ee.ImageCollection(dataset_id).filterDate(f'{year}-01-01', f'{year+1}-01-01')
            if country:
                collection = collection.filterBounds(country)
            image = collection.select(band).first()

        if image is None:
            print(f"⚠️ No image found for {dataset_id} year {year}")
            return None

        return ee.Image(image)

    except Exception as e:
        print(f"Error loading {dataset_id} for {year}: {e}")
        return None

# Initialize map
Map = geemap.Map()
Map.add_basemap("OpenStreetMap")
Map.add_draw_control()

# Dropdown for dataset
dataset_dropdown = widgets.Dropdown(
    options=[(v["label"], k) for k, v in datasets.items()],
    value="WorldPop/GP/100m/pop",
    description="Dataset:"
)

# SelectionSlider for year
year_slider = widgets.SelectionSlider(
    options=datasets["WorldPop/GP/100m/pop"]["years"],
    value=2020,
    description="Year:",
    layout=widgets.Layout(width="95%"),
    continuous_update=False
)

# Add legend
def add_legend():
    Map.add_legend(
        title="Population Count",
        labels=legend_labels,
        colors=legend_colors,
        position='bottomright'
    )

add_legend()

# Countries collection
countries = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017")

# Button handler
def get_population_data(b):
    with output:
        output.clear_output()
        print("🔍 Processing the selected area...")

        drawn_features = Map.draw_features
        if not drawn_features:
            print("⚠️ Please draw a polygon on the map.")
            return

        geometry = ee.Geometry(drawn_features[-1].geometry())
        Map.centerObject(geometry, zoom=9)
        Map.layers = Map.layers[:1]

        dataset_id = dataset_dropdown.value
        dataset_info = datasets[dataset_id]
        selected_year = year_slider.value

        if selected_year not in dataset_info["years"]:
            print(f"⚠️ {dataset_info['label']} does not support year {selected_year}.")
            return

        if dataset_id == "WorldPop/GP/100m/pop":
            matching_countries = countries.filterBounds(geometry)
            num_countries = len(matching_countries.getInfo()['features'])
            print(f"✅ Number of countries found: {num_countries}")

            country_images = []

            for country in matching_countries.getInfo()['features']:
                country_geometry = ee.Geometry(country['geometry'])
                image = load_dataset_by_year(dataset_id, dataset_info["band"], selected_year, country=country_geometry)
                if image:
                    country_images.append(image)

            if not country_images:
                print("⚠️ No valid population data for the selected countries.")
                return

            combined_image = ee.ImageCollection(country_images).sum()
            clipped = combined_image.clip(geometry)
            masked = clipped.updateMask(clipped.gt(0))
            classified = classify_image(masked)

            Map.addLayer(classified, vis_params, f"{dataset_info['label']} ({selected_year})")

            stats = combined_image.reduceRegion(
                reducer=ee.Reducer.sum(),
                geometry=geometry,
                scale=1000,
                maxPixels=1e12
            ).getInfo()

            pop_total = stats.get(dataset_info["band"], None)
            if pop_total is not None:
                print(f"✅ {dataset_info['label']} ({selected_year}) - Total population: {pop_total:.2f}")
            else:
                print("⚠️ No population data in the selected region.")

        else:
            image = load_dataset_by_year(dataset_id, dataset_info["band"], selected_year)
            if image is None:
                print("⚠️ Failed to load image.")
                return

            clipped = image.clip(geometry)
            masked = clipped.updateMask(clipped.gt(0))
            classified = classify_image(masked)

            Map.addLayer(classified, vis_params, f"{dataset_info['label']} ({selected_year})")

            stats = image.reduceRegion(
                reducer=ee.Reducer.sum(),
                geometry=geometry,
                scale=1000,
                maxPixels=1e12
            ).getInfo()

            pop_total = stats.get(dataset_info["band"], None)
            if pop_total is not None:
                print(f"✅ {dataset_info['label']} ({selected_year}) - Total population: {pop_total:.2f}")
            else:
                print("⚠️ No population data in the selected region.")

# Button widget
button = widgets.Button(description="Get Population Data")
button.on_click(get_population_data)

# Slider event connection
def on_slider_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        get_population_data(None)

year_slider.observe(on_slider_change)

# Dataset dropdown handler to update year slider
def update_slider(change):
    dataset_id = change["new"]
    years = datasets[dataset_id]["years"]
    year_slider.options = years
    year_slider.value = years[-1]

dataset_dropdown.observe(update_slider, names="value")

# Initial year slider setup
initial_years = datasets[dataset_dropdown.value]["years"]
year_slider.options = initial_years
year_slider.value = initial_years[-1]

# Display UI and Map
ui = widgets.VBox([dataset_dropdown, year_slider, button, output])
display(ui)
display(Map)


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [3]:
import ee 
import geemap
import ipywidgets as widgets
import matplotlib.pyplot as plt
import io
from IPython.display import display, Image
import matplotlib.ticker as mticker

# Initialize Earth Engine
ee.Initialize()

# Create output widget
output = widgets.Output()

# Dataset dictionary
datasets = {
    "CIESIN/GPWv411/GPW_UNWPP-Adjusted_Population_Density": {
        "label": "GPW v4.11 Population Density",
        "band": "unwpp-adjusted_population_density",
        "years": [2000, 2005, 2010, 2015, 2020]
    },
    "LandScan_Global": {
        "label": "LandScan Global",
        "band": "b1",
        "years": [2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021],
        "dynamic_id": "projects/sat-io/open-datasets/ORNL/LANDSCAN_GLOBAL/landscan-global-{year}"
    },
    "WorldPop/GP/100m/pop": {
        "label": "WorldPop 100m",
        "band": "population",
        "years": [2010, 2015, 2020]
    }
}

# Visualization parameters
bin_thresholds = [1, 6, 26, 51, 101, 501, 2501, 5001, 185000]
legend_labels = ['1-5', '6-25', '26-50', '51-100', '101-500', '501-2500', '2501-5000', '5001-185000']
legend_colors = ['#ffffcc', '#fff775', '#fed976', '#feb24c', '#fd8d3c', '#f03b20', '#bd0026', '#800026']

vis_params = {
    'min': 1,
    'max': len(legend_colors),
    'palette': legend_colors
}

def classify_image(image):
    bins = bin_thresholds
    classified = image.gt(bins[0]).And(image.lte(bins[1])).multiply(1)
    for i in range(1, len(bins) - 1):
        classified = classified.where(image.gt(bins[i]).And(image.lte(bins[i + 1])), i + 1)
    return classified

def load_dataset_by_year(dataset_id, band, year, country=None):
    try:
        if dataset_id == "LandScan_Global":
            image_id = datasets[dataset_id]["dynamic_id"].format(year=year)
            image = ee.Image(image_id).select(band)
        else:
            collection = ee.ImageCollection(dataset_id).filterDate(f'{year}-01-01', f'{year+1}-01-01')
            if country:
                collection = collection.filterBounds(country)
            image = collection.select(band).first()

        if image is None:
            print(f"⚠️ No image found for {dataset_id} year {year}")
            return None

        return ee.Image(image)

    except Exception as e:
        print(f"Error loading {dataset_id} for {year}: {e}")
        return None

def plot_population_comparison(pop_values, pop_labels, selected_year):
    plt.figure(figsize=(8, 5))
    bars = plt.bar(pop_labels, pop_values, color=['#4c72b0', '#55a868', '#c44e52'])
    plt.title(f"Total Population Comparison by Dataset ({selected_year})", fontsize=14)
    plt.xlabel("Population Dataset", fontsize=12)
    plt.xticks(rotation=25, ha='right')

    # Remove y-axis
    plt.gca().axes.get_yaxis().set_visible(False)

    # Add value labels on top of bars
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width() / 2, height, f'{int(height):,}', 
                 ha='center', va='bottom', fontsize=10)

    plt.tight_layout()
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    display(Image(buf.read()))
    plt.close()

# Initialize map
Map = geemap.Map()
Map.add_basemap("OpenStreetMap")
Map.add_draw_control()

# Dropdown for dataset
dataset_dropdown = widgets.Dropdown(
    options=[(v["label"], k) for k, v in datasets.items()],
    value="WorldPop/GP/100m/pop",
    description="Dataset:"
)

# SelectionSlider for year
year_slider = widgets.SelectionSlider(
    options=datasets["WorldPop/GP/100m/pop"]["years"],
    value=2020,
    description="Year:",
    layout=widgets.Layout(width="95%"),
    continuous_update=False
)

# Add legend
def add_legend():
    Map.add_legend(
        title="Population Count",
        labels=legend_labels,
        colors=legend_colors,
        position='bottomright'
    )

add_legend()

# Countries collection, loads a global countries dataset from Google Earth Engine's data catalog
countries = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017")

# Button handler
def get_population_data(b):
    with output:
        output.clear_output()
        print("🔍 Processing the selected area...")

        drawn_features = Map.draw_features
        if not drawn_features:
            print("⚠️ Please draw a polygon on the map.")
            return

        geometry = ee.Geometry(drawn_features[-1].geometry())
        Map.centerObject(geometry, zoom=9)
        Map.layers = Map.layers[:1]

        selected_year = year_slider.value

        pop_totals = []
        pop_labels = []
        
        for dataset_id, dataset_info in datasets.items():
            if selected_year not in dataset_info["years"]:
                print(f"⚠️ {dataset_info['label']} does not support year {selected_year}. Skipping.")
                continue
            
            if dataset_id == "WorldPop/GP/100m/pop":
                matching_countries = countries.filterBounds(geometry)
                country_images = []

                for country in matching_countries.getInfo()['features']:
                    country_geometry = ee.Geometry(country['geometry'])
                    image = load_dataset_by_year(dataset_id, dataset_info["band"], selected_year, country=country_geometry)
                    if image:
                        country_images.append(image)

                if not country_images:
                    print(f"⚠️ No valid population data for {dataset_info['label']} in the selected area.")
                    continue

                combined_image = ee.ImageCollection(country_images).sum()
                clipped = combined_image.clip(geometry)
                masked = clipped.updateMask(clipped.gt(0))
                classified = classify_image(masked)

                Map.addLayer(classified, vis_params, f"{dataset_info['label']} ({selected_year})")

                stats = combined_image.reduceRegion(
                    reducer=ee.Reducer.sum(),
                    geometry=geometry,
                    scale=1000,
                    maxPixels=1e12
                ).getInfo()

                pop_total = stats.get(dataset_info["band"], None)
            elif dataset_id == "LandScan_Global":
                image = load_dataset_by_year(dataset_id, dataset_info["band"], selected_year)
                if image is None:
                    print(f"⚠️ Failed to load image for {dataset_info['label']}.")
                    continue

                clipped = image.clip(geometry)
                masked = clipped.updateMask(clipped.gt(0))
                classified = classify_image(masked)

                Map.addLayer(classified, vis_params, f"{dataset_info['label']} ({selected_year})")

                stats = image.reduceRegion(
                    reducer=ee.Reducer.sum(),
                    geometry=geometry,
                    scale=1000,
                    maxPixels=1e12
                ).getInfo()

                pop_total = stats.get(dataset_info["band"], None)
            else:
                collection = ee.ImageCollection(dataset_id).filterDate(f'{selected_year}-01-01', f'{selected_year+1}-01-01')
                image = collection.select(dataset_info["band"]).first()
                if image is None:
                    print(f"⚠️ No image found for {dataset_info['label']} in {selected_year}.")
                    continue

                clipped = image.clip(geometry)
                masked = clipped.updateMask(clipped.gt(0))
                classified = classify_image(masked)

                Map.addLayer(classified, vis_params, f"{dataset_info['label']} ({selected_year})")

                stats = image.reduceRegion(
                    reducer=ee.Reducer.sum(),
                    geometry=geometry,
                    scale=1000,
                    maxPixels=1e12
                ).getInfo()

                pop_total = stats.get(dataset_info["band"], None)
             
            if pop_total is not None:
                print(f"✅ {dataset_info['label']} ({selected_year}) - Total population: {pop_total:.2f}")
                pop_totals.append(pop_total)
                pop_labels.append(dataset_info["label"])
            else:
                print(f"⚠️ No population data in the selected region for {dataset_info['label']}.")

        if pop_totals:
            plot_population_comparison(pop_totals, pop_labels, selected_year)

# Button widget
button = widgets.Button(description="Get Population Data")
button.on_click(get_population_data)

# Removed automatic update when slider moves
# year_slider.observe(on_slider_change)

# Dataset dropdown handler to update year slider
def update_slider(change):
    dataset_id = change["new"]
    years = datasets[dataset_id]["years"]
    year_slider.options = years
    year_slider.value = years[-1]

dataset_dropdown.observe(update_slider, names="value")

# Initial year slider setup
initial_years = datasets[dataset_dropdown.value]["years"]
year_slider.options = initial_years
year_slider.value = initial_years[-1]

# Display UI and Map
ui = widgets.VBox([dataset_dropdown, year_slider, button, output])
display(ui)
display(Map)


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [ ]:
pip install -U geemap


In [ ]:
print ('Hello World')

In [ ]:
!pip install geemap

In [4]:
import ee
ee.Authenticate()
ee.Initialize()

In [7]:
import ee
import geemap
import ipywidgets as widgets

# Initialize Earth Engine
ee.Initialize()

# Create output widget
output = widgets.Output()



# Visualization parameters
bin_thresholds = [1, 6, 26, 51, 101, 501, 2501, 5001, 185000]
legend_labels = ['1-5', '6-25', '26-50', '51-100', '101-500', '501-2500', '2501-5000', '5001-185000']
legend_colors = ['#ffffcc', '#fff775', '#fed976', '#feb24c', '#fd8d3c', '#f03b20', '#bd0026', '#800026']

vis_params = {
    'min': 1,
    'max': len(legend_colors),
    'palette': legend_colors
}

def classify_image(image):
    bins = bin_thresholds
    classified = image.gt(bins[0]).And(image.lte(bins[1])).multiply(1)
    for i in range(1, len(bins) - 1):
        classified = classified.where(image.gt(bins[i]).And(image.lte(bins[i + 1])), i + 1)
    return classified

def load_dataset_by_year(dataset_id, band, year, country=None):
    try:
        if dataset_id == "LandScan_Global":
            image_id = datasets[dataset_id]["dynamic_id"].format(year=year)
            image = ee.Image(image_id).select(band)
        else:
            # For other datasets (collections), filter by date and select first image
            collection = ee.ImageCollection(dataset_id).filterDate(f'{year}-01-01', f'{year+1}-01-01')
            if country:
                collection = collection.filterBounds(country)
            image = collection.select(band).first()

        if image is None:
            print(f"⚠️ No image found for {dataset_id} year {year}")
            return None

        return ee.Image(image)

    except Exception as e:
        print(f"Error loading {dataset_id} for {year}: {e}")
        return None

# Initialize map
Map = geemap.Map()
Map.add_basemap("OpenStreetMap")
Map.add_draw_control()

# Dropdown for dataset
dataset_dropdown = widgets.Dropdown(
    options=[(v["label"], k) for k, v in datasets.items()],
    value="WorldPop/GP/100m/pop",
    description="Dataset:"
)

# SelectionSlider for year
year_slider = widgets.SelectionSlider(
    options=datasets["WorldPop/GP/100m/pop"]["years"],
    value=2020,
    description="Year:",
    layout=widgets.Layout(width="95%"),
    continuous_update=False
)

# Add legend
def add_legend():
    Map.add_legend(
        title="Population Count",
        labels=legend_labels,
        colors=legend_colors,
        position='bottomright'
    )

add_legend()

# Countries collection
countries = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017")

# Button handler
def get_population_data(b):
    with output:
        output.clear_output()
        print("🔍 Processing the selected area...")

        drawn_features = Map.draw_features
        if not drawn_features:
            print("⚠️ Please draw a polygon on the map.")
            return

        geometry = ee.Geometry(drawn_features[-1].geometry())
        Map.centerObject(geometry, zoom=9)
        Map.layers = Map.layers[:1]

        dataset_id = dataset_dropdown.value
        dataset_info = datasets[dataset_id]
        selected_year = year_slider.value

        if selected_year not in dataset_info["years"]:
            print(f"⚠️ {dataset_info['label']} does not support year {selected_year}.")
            return

        if dataset_id == "WorldPop/GP/100m/pop":
            matching_countries = countries.filterBounds(geometry)
            num_countries = len(matching_countries.getInfo()['features'])
            print(f"✅ Number of countries found: {num_countries}")

            country_images = []

            for country in matching_countries.getInfo()['features']:
                country_geometry = ee.Geometry(country['geometry'])
                image = load_dataset_by_year(dataset_id, dataset_info["band"], selected_year, country=country_geometry)
                if image:
                    country_images.append(image)

            if not country_images:
                print("⚠️ No valid population data for the selected countries.")
                return

            combined_image = ee.ImageCollection(country_images).sum()
            clipped = combined_image.clip(geometry)
            masked = clipped.updateMask(clipped.gt(0))
            classified = classify_image(masked)

            Map.addLayer(classified, vis_params, f"{dataset_info['label']} ({selected_year})")

            stats = combined_image.reduceRegion(
                reducer=ee.Reducer.sum(),
                geometry=geometry,
                scale=1000,
                maxPixels=1e12
            ).getInfo()

            pop_total = stats.get(dataset_info["band"], None)
            if pop_total is not None:
                print(f"✅ {dataset_info['label']} ({selected_year}) - Total population: {pop_total:.2f}")
            else:
                print("⚠️ No population data in the selected region.")

        else:
            image = load_dataset_by_year(dataset_id, dataset_info["band"], selected_year)
            if image is None:
                print("⚠️ Failed to load image.")
                return

            clipped = image.clip(geometry)
            masked = clipped.updateMask(clipped.gt(0))
            classified = classify_image(masked)

            Map.addLayer(classified, vis_params, f"{dataset_info['label']} ({selected_year})")

            stats = image.reduceRegion(
                reducer=ee.Reducer.sum(),
                geometry=geometry,
                scale=1000,
                maxPixels=1e12
            ).getInfo()

            pop_total = stats.get(dataset_info["band"], None)
            if pop_total is not None:
                print(f"✅ {dataset_info['label']} ({selected_year}) - Total population: {pop_total:.2f}")
            else:
                print("⚠️ No population data in the selected region.")

# Button widget
button = widgets.Button(description="Get Population Data")
button.on_click(get_population_data)

# Slider event connection
def on_slider_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        get_population_data(None)

year_slider.observe(on_slider_change)

# Dataset dropdown handler to update year slider
def update_slider(change):
    dataset_id = change["new"]
    years = datasets[dataset_id]["years"]
    year_slider.options = years
    year_slider.value = years[-1]

dataset_dropdown.observe(update_slider, names="value")

# Initial year slider setup
initial_years = datasets[dataset_dropdown.value]["years"]
year_slider.options = initial_years
year_slider.value = initial_years[-1]

# Display UI and Map
ui = widgets.VBox([dataset_dropdown, year_slider, button, output])
display(ui)
display(Map)

EEException: Please authorize access to your Earth Engine account by running

earthengine authenticate

in your command line, or ee.Authenticate() in Python, and then retry.